SHUFFLENETPRETRAINED

This script is very similar to the Dataloader Training script, however it uses one of the premade models within Pytorch. This makes it a useful example for how to use these models.

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2

Here, the model is loaded in from Pytorch. This can be set to either pretrained or not. A list of these models can be found here https://pytorch.org/docs/stable/torchvision/models.html

In [ ]:
#model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=False)
model = torch.hub.load('pytorch/vision:v0.6.0', 'shufflenet_v2_x1_0', pretrained=False)
model.eval()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

model.to(device)

In [ ]:
base = 'C:\\Users\\zshal\\Documents\\Pytorch\\Images\\Images\\v3'

preprocess = transforms.Compose([
    # transforms.Resize(256),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data = ImageFolder(root=base, transform=preprocess)


trainimages = DataLoader(data, batch_size=4, shuffle=True, num_workers=0)
testimages = DataLoader(data, batch_size=4, shuffle=False, num_workers=0)

In [ ]:
classes = ('pothole', 'no pothole')


PATH = './shuffle.pth'

model.load_state_dict(torch.load(PATH))

dataiter = iter(trainimages)
images, labels = dataiter.next()

In [ ]:
def imshow(img):

    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [ ]:
# print images
imshow(torchvision.utils.make_grid(images))
images.to(device)
labels.to(device)
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
outputs = model(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for data in testimages:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in testimages:
        images, labels = data[0].to(device), data[1].to(device)  
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(2):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            
for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))